function ClickConnect(){
console.log("Working"); 
document.querySelector("colab-toolbar-button#connect").click() 
}
setInterval(ClickConnect,60000)

In [ ]:
%%capture
!pip install sentencepiece
!pip install transformers
!pip install wandb

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Importing stock libraries
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler

# Importing the T5 modules from huggingface/transformers
#from transformers import T5Tokenizer, T5ForConditionalGeneration
from transformers import BartForConditionalGeneration, BartTokenizer

# WandB – Import the wandb library
import wandb

In [ ]:
!nvidia-smi

Sat Dec 12 23:16:18 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.45.01    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P8    10W /  70W |     10MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# # Setting up the device for GPU usage
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

# Preparing for TPU usage
# import torch_xla
# import torch_xla.core.xla_model as xm
# device = xm.xla_device()

In [ ]:
# Login to wandb to log the model run and all the parameters
!wandb login

wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
# Creating a custom dataset for reading the dataframe and loading it into the dataloader to pass it to the neural network at a later stage for finetuning the model and to prepare it for predictions

class CustomDataset(Dataset):

    def __init__(self, dataframe, tokenizer, source_len, summ_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.source_len = source_len
        self.summ_len = summ_len
        self.text = self.data.text
        self.ctext = self.data.ctext

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        ctext = str(self.ctext[index])
        ctext = ' '.join(ctext.split())

        text = str(self.text[index])
        text = ' '.join(text.split())

        source = self.tokenizer.batch_encode_plus([ctext], max_length=self.source_len, pad_to_max_length=True,return_tensors='pt')
        target = self.tokenizer.batch_encode_plus([text], max_length=self.summ_len, pad_to_max_length=True,return_tensors='pt')

        source_ids = source['input_ids'].squeeze()
        source_mask = source['attention_mask'].squeeze()
        target_ids = target['input_ids'].squeeze()
        target_mask = target['attention_mask'].squeeze()

        return {
            'source_ids': source_ids.to(dtype=torch.long), 
            'source_mask': source_mask.to(dtype=torch.long), 
            'target_ids': target_ids.to(dtype=torch.long),
            'target_ids_y': target_ids.to(dtype=torch.long)
        }

validation loss reference: https://mccormickml.com/2019/07/22/BERT-fine-tuning/

In [ ]:
def get_validation_loss(tokenizer, model, val_loader, device):
    model.eval()
    total_eval_loss = 0
    with torch.no_grad():
        for _,data in enumerate(val_loader, 0):
            y = data['target_ids'].to(device, dtype = torch.long)
            y_ids = y[:, :-1].contiguous()
            labels = y[:, 1:].clone().detach()
            labels[y[:, 1:] == tokenizer.pad_token_id] = -100
            ids = data['source_ids'].to(device, dtype = torch.long)
            mask = data['source_mask'].to(device, dtype = torch.long)

            outputs = model(input_ids = ids, attention_mask = mask, decoder_input_ids=y_ids, labels=labels)
            loss = outputs[0]
            total_eval_loss += loss.item()
      
    avg_loss = total_eval_loss / len(val_loader)
    return avg_loss
            
            #if _%10 == 0:
            #    wandb.log({"Validation Loss": loss.item()})

            #if _%500==0:
            #    print(f'Epoch: {epoch}, Loss:  {loss.item()}')

In [ ]:
# Creating the training function. This will be called in the main function. It is run depending on the epoch value.
# The model is put into train mode and then we wnumerate over the training loader and passed to the defined network 

def train(epoch, tokenizer, model, device, loader, val_loader, optimizer, output_dir):
    model.train()
    for _,data in enumerate(loader, 0):
        y = data['target_ids'].to(device, dtype = torch.long)
        y_ids = y[:, :-1].contiguous()
        labels = y[:, 1:].clone().detach()
        labels[y[:, 1:] == tokenizer.pad_token_id] = -100
        ids = data['source_ids'].to(device, dtype = torch.long)
        mask = data['source_mask'].to(device, dtype = torch.long)

        outputs = model(input_ids = ids, attention_mask = mask, decoder_input_ids=y_ids, labels=labels)
        loss = outputs[0]
        
        if _ % 250 == 0:
            val_loss = get_validation_loss(tokenizer, model, val_loader, device)
            wandb.log({"Step": _, "Training Loss": loss.item(), "Validation Loss": val_loss})
        elif _ % 10 == 0:
            wandb.log({"Step": _, "Training Loss": loss.item()})

        if _ % 500==0:
            #val_loss = get_validation_loss(tokenizer, model, val_loader, device)
            print(f'Epoch: {epoch}, Step: {_}, Train Loss:  {loss.item()}, Validation Loss: {val_loss}')
            model.save_pretrained(output_dir + 'model_epoch{}_step{}'.format(epoch, _))
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # xm.optimizer_step(optimizer)
        # xm.mark_step()

In [ ]:
def test(epoch, tokenizer, model, device, loader):
    model.eval()
    predictions = []
    actuals = []
    with torch.no_grad():
        for _, data in enumerate(loader, 0):
            y = data['target_ids'].to(device, dtype = torch.long)
            ids = data['source_ids'].to(device, dtype = torch.long)
            mask = data['source_mask'].to(device, dtype = torch.long)

            generated_ids = model.generate(
                input_ids = ids,
                attention_mask = mask, 
                max_length=54,   # should this be 54?
                num_beams=2,
                repetition_penalty=2.5, 
                length_penalty=1.0, 
                early_stopping=True
                )
            preds = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in generated_ids]
            target = [tokenizer.decode(t, skip_special_tokens=True, clean_up_tokenization_spaces=True) for t in y]
            if _%100==0:
                print(f'Completed {_}')

            predictions.extend(preds)
            actuals.extend(target)
    return predictions, actuals

In [ ]:
# WandB – Initialize a new run
wandb.init(project="transformers_summarization")

# WandB – Config is a variable that holds and saves hyperparameters and inputs
# Defining some key variables that will be used later on in the training  
config = wandb.config          # Initialize config
config.TRAIN_BATCH_SIZE = 25    # input batch size for training (default: 64)
config.VALID_BATCH_SIZE = 25    # input batch size for validation (default: 1000)
config.TEST_BATCH_SIZE = 25    # input batch size for testing (default: 1000)
config.TRAIN_EPOCHS = 3        # number of epochs to train (default: 10)
config.VAL_EPOCHS = 1 
config.LEARNING_RATE = 1e-4    # learning rate (default: 0.01)
config.SEED = 42               # random seed (default: 42)
config.MAX_LEN = 200
config.SUMMARY_LEN = 54
config.OUTPUT_DIR = '/content/drive/MyDrive/AML_workshop/bart-checkpoint/121111cont/'

# Set random seeds and deterministic pytorch for reproducibility
torch.manual_seed(config.SEED) # pytorch random seed
np.random.seed(config.SEED) # numpy random seed
torch.backends.cudnn.deterministic = True

# tokenzier for encoding the text
tokenizer = BartTokenizer.from_pretrained("facebook/bart-base")

# Creation of Dataset and Dataloader
train_dataset = pd.read_csv('/content/drive/MyDrive/AML_workshop/train_set_large.csv')
val_dataset = pd.read_csv('/content/drive/MyDrive/AML_workshop/val_set_large.csv')
test_dataset = pd.read_csv('/content/drive/MyDrive/AML_workshop/test_set_2000.csv')
train_dataset.columns = ['id', 'ctext','text']
val_dataset.columns = ['id', 'ctext','text']
test_dataset.columns = ['id', 'ctext','text']
train_dataset.ctext = 'summarize: ' + train_dataset.ctext
val_dataset.ctext = 'summarize: ' + val_dataset.ctext
test_dataset.ctext = 'summarize: ' + test_dataset.ctext

# Creating the Training and Validation dataset for further creation of Dataloader
training_set = CustomDataset(train_dataset, tokenizer, config.MAX_LEN, config.SUMMARY_LEN)
val_set = CustomDataset(val_dataset, tokenizer, config.MAX_LEN, config.SUMMARY_LEN)
test_set = CustomDataset(test_dataset, tokenizer, config.MAX_LEN, config.SUMMARY_LEN)

# Defining the parameters for creation of dataloaders
train_params = {
    'batch_size': config.TRAIN_BATCH_SIZE,
    'shuffle': True,
    'num_workers': 0
    }

val_params = {
    'batch_size': config.VALID_BATCH_SIZE,
    'shuffle': False,
    'num_workers': 0
    }

test_params = {
    'batch_size': config.TEST_BATCH_SIZE,
    'shuffle': False,
    'num_workers': 0
    }

# Creation of Dataloaders for testing and validation. This will be used down for training and validation stage for the model.
training_loader = DataLoader(training_set, **train_params)
val_loader = DataLoader(val_set, **val_params)
test_loader = DataLoader(test_set, **test_params)

# Defining the model. We are using t5-base model and added a Language model layer on top for generation of Summary. 
# Further this model is sent to device (GPU/TPU) for using the hardware.
model = BartForConditionalGeneration.from_pretrained('/content/drive/MyDrive/AML_workshop/bart-checkpoint/121111/model_step_1epoch')
model = model.to(device)

# Defining the optimizer that will be used to tune the weights of the network in the training session. 
optimizer = torch.optim.Adam(params =  model.parameters(), lr=config.LEARNING_RATE)

# Log metrics with wandb
wandb.watch(model, log="all")
# Training loop
print('Initiating Fine-Tuning for the model on our dataset')

for epoch in range(config.TRAIN_EPOCHS):
    train(epoch, tokenizer, model, device, training_loader, val_loader, optimizer, config.OUTPUT_DIR)

# Validation loop and saving the resulting file with predictions and acutals in a dataframe.
# Saving the dataframe as predictions.csv
print('Now generating summaries on our fine tuned model for the test dataset and saving it in a dataframe')
for epoch in range(config.VAL_EPOCHS):
    predictions, actuals = test(epoch, tokenizer, model, device, test_loader)
    final_df = pd.DataFrame({'Generated Text':predictions,'Actual Text':actuals})
    final_df.to_csv('/content/drive/MyDrive/AML_workshop/BART-finetune_pred_121111cont.csv')
    print('Output Files generated for review')

wandb: Currently logged in as: xuanamylin (use `wandb login --relogin` to force relogin)


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Initiating Fine-Tuning for the model on our dataset


/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:2142: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Epoch: 0, Step: 0, Train Loss:  2.3310232162475586, Validation Loss: 1.3948966123344442
Epoch: 0, Step: 500, Train Loss:  1.2546675205230713, Validation Loss: 1.3971092406951513
Epoch: 0, Step: 1000, Train Loss:  1.2128475904464722, Validation Loss: 1.3846706347966773
Epoch: 0, Step: 1500, Train Loss:  1.233654499053955, Validation Loss: 1.3825676519915742
Epoch: 0, Step: 2000, Train Loss:  1.2559758424758911, Validation Loss: 1.3850292638948343
Epoch: 0, Step: 2500, Train Loss:  1.2558677196502686, Validation Loss: 1.3866982069619582
Epoch: 0, Step: 3000, Train Loss:  1.109267234802246, Validation Loss: 1.3729378047657783
Epoch: 1, Step: 0, Train Loss:  2.0672478675842285, Validation Loss: 1.3527785224734612
Epoch: 1, Step: 500, Train Loss:  1.0977572202682495, Validation Loss: 1.359898311269251
Epoch: 1, Step: 1000, Train Loss:  1.107664704322815, Validation Loss: 1.352440585665947
Epoch: 1, Step: 1500, Train Loss:  1.2495150566101074, Validation Loss: 1.3436521098941485
Epoch: 1, St

Epoch: 0, Train Loss:  5.132025241851807, Validation Loss: 4.762231125021881
Epoch: 0, Train Loss:  1.799232840538025, Validation Loss: 1.963273718029341
Epoch: 0, Train Loss:  1.4732366800308228, Validation Loss: 1.589828784896357
Epoch: 0, Train Loss:  1.5044106245040894, Validation Loss: 1.5101558519502214
Epoch: 0, Train Loss:  1.5279278755187988, Validation Loss: 1.4768771204665665
Epoch: 0, Train Loss:  1.5081861019134521, Validation Loss: 1.4375688554141721
Epoch: 0, Train Loss:  1.334514856338501, Validation Loss: 1.4071523807440807

#Test Set Evaluation

In [ ]:
#model = BartForConditionalGeneration.from_pretrained('/content/drive/MyDrive/AML_workshop/bart-checkpoint/121111/model_epoch1_step3000')
model = model.to(device)
for epoch in range(config.VAL_EPOCHS):
    predictions, actuals = test(epoch, tokenizer, model, device, test_loader)
    final_df = pd.DataFrame({'Generated Text':predictions,'Actual Text':actuals})
    final_df.to_csv('/content/drive/MyDrive/AML_workshop/BART-finetune_pred_121111cont_3epoch_54.csv')
    print('Output Files generated for review')

/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:2142: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Completed 0
Output Files generated for review


In [ ]:
pred = pd.read_csv('/content/drive/MyDrive/AML_workshop/BART-finetune_pred_121111cont_3epoch.csv')

In [ ]:
pred.head()

,Unnamed: 0,Generated Text,Actual Text
0,0,1.There is a right parietal lobe mass present ...,Subacute infarction involving the right poster...
1,1,1.Interval progression of moderate effacement ...,"1.Over the interval, previously demonstrated m..."
2,2,1.No evidence of pulmonary embolism.2.Bilatera...,1.No evidence of pulmonary embolism.2.Centrilo...
3,3,Enteric tube seen curled on itself with tip ex...,No change in appearance of enteric tube as abo...
4,4,1. No evidence of acute intracranial hemorrhag...,1. No evidence of intracranial hemorrhage or m...


In [ ]:
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.bleu_score import SmoothingFunction
chencherry = SmoothingFunction()

def sentence_bleu_n(ref, hyp, weights):
  return sentence_bleu(references = [ref.split()], 
                       hypothesis = hyp.split(),
                       weights = weights,
                       smoothing_function = chencherry.method1)

# bleu1
pred['bleu1'] = pred[['Actual Text', 'Generated Text']].apply(lambda x: sentence_bleu_n(x[1], x[0], weights = [1,0,0,0]), axis=1)

# bleu2
pred['bleu2'] = pred[['Actual Text', 'Generated Text']].apply(lambda x: sentence_bleu_n(x[1], x[0], weights = [0,1,0,0]), axis=1)

print('bleu1: {}'.format(pred['bleu1'].mean()))
print('bleu2: {}'.format(pred['bleu2'].mean()))

bleu1: 0.3139187092358421
bleu2: 0.20471319815566838


In [ ]:
!pip install rouge
from rouge import Rouge 

# rouge1
pred['rouge1'] = pred[['Actual Text', 'Generated Text']].apply(lambda x: Rouge().get_scores(x[0], x[1])[0]['rouge-1']['f'], axis=1)

# rouge2
pred['rouge2'] = pred[['Actual Text', 'Generated Text']].apply(lambda x: Rouge().get_scores(x[0], x[1])[0]['rouge-2']['f'], axis=1)

print('rouge1: {}'.format(pred['rouge1'].mean()))
print('rouge2: {}'.format(pred['rouge2'].mean()))

rouge1: 0.4105500399130565
rouge2: 0.25743032005237954


In [ ]:
pred.head()

,Unnamed: 0,Generated Text,Actual Text,bleu1,bleu2,rouge1,rouge2
0,0,1.There is a right parietal lobe mass present ...,Subacute infarction involving the right poster...,0.368421,0.135135,0.379747,0.129870
1,1,1.Interval progression of moderate effacement ...,"1.Over the interval, previously demonstrated m...",0.319239,0.210408,0.358209,0.215385
2,2,1.No evidence of pulmonary embolism.2.Bilatera...,1.No evidence of pulmonary embolism.2.Centrilo...,0.725068,0.558545,0.813559,0.666667
3,3,Enteric tube seen curled on itself with tip ex...,No change in appearance of enteric tube as abo...,0.234853,0.006263,0.242424,0.000000
4,4,1. No evidence of acute intracranial hemorrhag...,1. No evidence of intracranial hemorrhage or m...,0.457143,0.294118,0.646154,0.412698


In [ ]:
final_df.head(15)

,Generated Text,Actual Text
0,1. Diffusion restriction in the right paracent...,Subacute infarction involving the right poster...
1,1. No interval new hemorrhage.2. Extensive vas...,"1.Over the interval, previously demonstrated m..."
2,1. No evidence of pulmonary embolus.2. Mild ce...,1.No evidence of pulmonary embolism.2.Centrilo...
3,No tube seen curled on itself with tip extendi...,No change in appearance of enteric tube as abo...
4,1. No evidence of intracranial hemorrhage.2. I...,1. No evidence of intracranial hemorrhage or m...
5,1. No evidence of metastatic disease.2. New me...,No change in the previously described left low...
6,1. No evidence of metastatic disease.2. Modera...,Diffuse colonic wall thickening with moderate ...
7,1. No evidence of bowel obstruction.2. Choleli...,1.Interval increase in peripancreatic fluid an...
8,No acute intracranial abnormality.,No specific evidence of infection or edema.
9,No pulmonary edema. No evidence of pneumonia.,No acute cardiopulmonary disease. Left lower l...


In [ ]:
test_dataset = pd.read_csv('/content/drive/MyDrive/AML_workshop/test_set_2000.csv', nrows = 200)
test_dataset.head(15)

,index,FINDINGS,IMPRESSION
0,0,There is diffusion restriction present involvi...,Subacute infarction involving the right poster...
1,1,"Over the interval, previously demonstrated mod...","1.Over the interval, previously demonstrated m..."
2,2,PULMONARY ARTERIES: Diagnostic quality examina...,1.No evidence of pulmonary embolism.2.Centrilo...
3,3,Enteric tube seen curled on itself with tip ex...,No change in appearance of enteric tube as abo...
4,4,No intracranial hemorrhage is identified. No i...,1. No evidence of intracranial hemorrhage or m...
5,5,CHEST:LUNGS AND PLEURA: Previously described 3...,No change in the previously described left low...
6,6,ABDOMEN: Lack of intravenous contrast limits e...,Diffuse colonic wall thickening with moderate ...
7,7,Note that the lack of oral contrast limits eva...,1.Interval increase in peripancreatic fluid an...
8,8,Heart size normal.Basilar opacities likely ate...,No specific evidence of infection or edema.
9,9,An ICD is noted with tips overlying the right ...,No acute cardiopulmonary disease. Left lower l...
